In [1]:
import pandas as pd
import numpy as np
import re
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.utils import class_weight
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import accuracy_score
import string
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import accuracy_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report
from gensim.models import Word2Vec
import datetime as dt  # Manipulasi data waktu dan tanggal
from nltk.tokenize import word_tokenize  # Tokenisasi teks

!pip install sastrawi
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory  # Stemming (penghilangan imbuhan kata) dalam bahasa Indonesia
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory  # Menghapus kata-kata berhenti dalam bahasa Indonesia

from wordcloud import WordCloud  # Membuat visualisasi berbentuk awan kata (word cloud) dari teks

import nltk  # Import pustaka NLTK (Natural Language Toolkit).
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer, PorterStemmer

import csv
import requests
from io import StringIO

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.7/209.7 kB 4.8 MB/s eta 0:00:00


In [2]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [3]:
!apt-get install git-lfs
!git lfs install
!git clone https://github.com/Sipa02/sentimen-analysis.git
%cd sentimen-analysis
!git lfs pull


df = pd.read_csv('ulasan_aplikasi_tokopedia.csv')

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
git-lfs is already the newest version (3.0.2-1ubuntu0.3).
0 upgraded, 0 newly installed, 0 to remove and 29 not upgraded.
Git LFS initialized.
Cloning into 'sentimen-analysis'...
remote: Enumerating objects: 18, done.
remote: Counting objects: 100% (18/18), done.
remote: Compressing objects: 100% (15/15), done.
remote: Total 18 (delta 2), reused 11 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (18/18), done.
Resolving deltas: 100% (2/2), done.
/content/sentimen-analysis


In [4]:
!pip freeze requirements.txt

absl-py==1.4.0
accelerate==1.3.0
aiohappyeyeballs==2.4.6
aiohttp==3.11.13
aiosignal==1.3.2
alabaster==1.0.0
albucore==0.0.23
albumentations==2.0.4
ale-py==0.10.2
altair==5.5.0
annotated-types==0.7.0
anyio==3.7.1
argon2-cffi==23.1.0
argon2-cffi-bindings==21.2.0
array_record==0.6.0
arviz==0.20.0
astropy==7.0.1
astropy-iers-data==0.2025.2.24.0.34.4
astunparse==1.6.3
atpublic==4.1.0
attrs==25.1.0
audioread==3.0.1
autograd==1.7.0
babel==2.17.0
backcall==0.2.0
beautifulsoup4==4.13.3
betterproto==2.0.0b6
bigframes==1.38.0
bigquery-magics==0.6.0
bleach==6.2.0
blinker==1.9.0
blis==0.7.11
blosc2==3.1.1
bokeh==3.6.3
Bottleneck==1.4.2
bqplot==0.12.44
branca==0.8.1
CacheControl==0.14.2
cachetools==5.5.2
catalogue==2.0.10
certifi==2025.1.31
cffi==1.17.1
chardet==5.2.0
charset-normalizer==3.4.1
chex==0.1.89
clarabel==0.10.0
click==8.1.8
cloudpathlib==0.20.0
cloudpickle==3.1.1
cmake==3.31.4
cmdstanpy==1.2.5
colorcet==3.1.0
colorlover==0.3.0
colour==0.1.5
community==1.0.0b1
confection==0.1.5
cons==0.4.

In [ ]:
df.tail()

,reviewId,userName,userImage,content,score,thumbsUpCount,reviewCreatedVersion,at,replyContent,repliedAt,appVersion
184495,b4edeaa9-47f2-4de8-ae3d-b05d921e2a42,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,Bagus dan lancar pesan apapun,5,0,3.186,2022-09-16 10:19:38,"Hai Toppers, Terima kasih untuk rating yang te...",2022-09-16 10:32:19,3.186
184496,dfd1ce31-d45a-4c54-8b90-91eeb1aae5af,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,sangat mudah skrng ada paylater hehe,5,0,3.250,2024-01-30 21:49:03,Terima kasih sudah mempercayakan Tokopedia seb...,2024-01-30 22:00:04,3.250
184497,1f366168-125d-4273-a5e5-72e3d0983a2e,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,dapet promo 1rb pas dibeli malah error tiap ha...,1,0,NaN,2023-11-10 23:26:30,"Hi Toppers, silakan sampaikan kritik/saran ata...",2023-11-10 23:40:03,NaN
184498,e5e787c4-a77e-41db-811a-a038c97aed7e,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,Bagus dan tingkatkan pelayanan,5,0,3.5,2018-11-05 08:10:56,NaN,NaN,3.5
184499,73fd496a-fb0e-4291-b1bb-c1d62b6193c3,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,"Cinta bgt sma tokped. sukses trs ya ped, lanca...",5,0,3.140,2021-09-06 01:11:20,NaN,NaN,3.140


In [ ]:
df.shape

(184500, 11)

In [5]:
df = df.dropna().drop_duplicates()

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 70189 entries, 0 to 184496
Data columns (total 11 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   reviewId              70189 non-null  object
 1   userName              70189 non-null  object
 2   userImage             70189 non-null  object
 3   content               70189 non-null  object
 4   score                 70189 non-null  int64 
 5   thumbsUpCount         70189 non-null  int64 
 6   reviewCreatedVersion  70189 non-null  object
 7   at                    70189 non-null  object
 8   replyContent          70189 non-null  object
 9   repliedAt             70189 non-null  object
 10  appVersion            70189 non-null  object
dtypes: int64(2), object(9)
memory usage: 6.4+ MB


In [7]:
df.shape

(70189, 11)

In [8]:
def cleaningText(text):
    text = re.sub(r'@[A-Za-z0-9]+', '', text)  # Menghapus mention
    text = re.sub(r'#[A-Za-z0-9]+', '', text)  # Menghapus hashtag
    text = re.sub(r'RT[\s]', '', text)  # Menghapus RT
    text = re.sub(r"http\S+", '', text)  # Menghapus link
    text = re.sub(r'[0-9]+', '', text)  # Menghapus angka
    text = re.sub(r'[^\w\s]', '', text)  # Menghapus karakter selain huruf dan angka
    text = text.replace('\n', ' ')  # Mengganti baris baru dengan spasi
    text = text.translate(str.maketrans('', '', string.punctuation))  # Menghapus semua tanda baca
    text = text.strip()  # Menghapus spasi dari kiri dan kanan teks
    return text

def casefoldingText(text):  # Mengubah semua karakter menjadi huruf kecil
    return text.lower()

def tokenizingText(text):  # Memecah teks menjadi daftar token
    return word_tokenize(text)

def filteringText(text):  # Menghapus stopwords dalam teks
    listStopwords = set(stopwords.words('indonesian'))
    listStopwords1 = set(stopwords.words('english'))
    listStopwords.update(listStopwords1)
    tambahan_stopwords = {
        'iya', 'yaa', 'gak', 'nya', 'na', 'sih', 'ku', "di", "ga", "ya", "gaa",
        "loh", "kah", "woi", "woii", "woy", "banget", "nih", "dong", "deh",
        "kan", "aja", "tuh", "kayak", "gitu", "begitu", "cuma", "doang",
        "gue", "lu", "loe", "gw", "loe", "oke", "ok", "bro", "sis", "cewek",
        "cowok", "men", "sob", "cuy", "lah", "eh", "hmm", "huhu", "hahaha",
        "wkwk", "wkwkwk", "haha", "hehe", "hoho"
    }

    listStopwords.update(tambahan_stopwords)
    return [word for word in text if word.lower() not in listStopwords]

def stemmingText(text): # Mengurangi kata ke bentuk dasarnya yang menghilangkan imbuhan awalan dan akhiran atau ke akar kata
    # Membuat objek stemmer
    factory = StemmerFactory()
    stemmer = factory.create_stemmer()

    # Memecah teks menjadi daftar kata
    words = text.split()

    # Menerapkan stemming pada setiap kata dalam daftar
    stemmed_words = [stemmer.stem(word) for word in words]

    # Menggabungkan kata-kata yang telah distem
    stemmed_text = ' '.join(stemmed_words)

    return stemmed_text

def toSentence(list_words):  # Mengubah daftar kata menjadi kalimat
    return ' '.join(list_words)


In [9]:
slangwords = {"@": "di", "abis": "habis", "wtb": "beli", "masi": "masih", "wts": "jual", "wtt": "tukar", "bgt": "banget", "maks": "maksimal", "min": "minimal"}
def fix_slangwords(text):
    words = text.split()
    fixed_words = []

    for word in words:
        if word.lower() in slangwords:
            fixed_words.append(slangwords[word.lower()])
        else:
            fixed_words.append(word)

    fixed_text = ' '.join(fixed_words)
    return fixed_text

In [10]:
# Membersihkan teks dan menyimpannya di kolom 'text_clean'
df['text_clean'] = df['content'].apply(cleaningText)

# Mengubah huruf dalam teks menjadi huruf kecil dan menyimpannya di 'text_casefoldingText'
df['text_casefoldingText'] = df['text_clean'].apply(casefoldingText)

# Mengganti kata-kata slang dengan kata-kata standar dan menyimpannya di 'text_slangwords'
df['text_slangwords'] = df['text_casefoldingText'].apply(fix_slangwords)

# Memecah teks menjadi token (kata-kata) dan menyimpannya di 'text_tokenizingText'
df['text_tokenizingText'] = df['text_slangwords'].apply(tokenizingText)

# Menghapus kata-kata stop (kata-kata umum) dan menyimpannya di 'text_stopword'
df['text_stopword'] = df['text_tokenizingText'].apply(filteringText)

# Menggabungkan token-token menjadi kalimat dan menyimpannya di 'text_akhir'
df['text_akhir'] = df['text_stopword'].apply(toSentence)

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


In [11]:
# Membaca data kamus kata-kata positif dari GitHub
lexicon_positive = dict()

response = requests.get('https://raw.githubusercontent.com/angelmetanosaa/dataset/main/lexicon_positive.csv')
# Mengirim permintaan HTTP untuk mendapatkan file CSV dari GitHub

if response.status_code == 200:
    # Jika permintaan berhasil
    reader = csv.reader(StringIO(response.text), delimiter=',')
    # Membaca teks respons sebagai file CSV menggunakan pembaca CSV dengan pemisah koma

    for row in reader:
        # Mengulangi setiap baris dalam file CSV
        lexicon_positive[row[0]] = int(row[1])
        # Menambahkan kata-kata positif dan skornya ke dalam kamus lexicon_positive
else:
    print("Failed to fetch positive lexicon data")

# Membaca data kamus kata-kata negatif dari GitHub
lexicon_negative = dict()

response = requests.get('https://raw.githubusercontent.com/angelmetanosaa/dataset/main/lexicon_negative.csv')
# Mengirim permintaan HTTP untuk mendapatkan file CSV dari GitHub

if response.status_code == 200:
    # Jika permintaan berhasil
    reader = csv.reader(StringIO(response.text), delimiter=',')
    # Membaca teks respons sebagai file CSV menggunakan pembaca CSV dengan pemisah koma

    for row in reader:
        # Mengulangi setiap baris dalam file CSV
        lexicon_negative[row[0]] = int(row[1])
        # Menambahkan kata-kata negatif dan skornya dalam kamus lexicon_negative
else:
    print("Failed to fetch negative lexicon data")

In [12]:
def sentiment_analysis_lexicon_indonesia(text):
    score = 0
    for word in text:
        if (word in lexicon_positive):
            score = score + lexicon_positive[word]
    for word in text:
        if (word in lexicon_negative):
            score = score + lexicon_negative[word]

    # Perubahan di sini:
    if (score > 0):
        polarity = 'positive'
    elif (score < 0):
        polarity = 'negative'
    else:  # Jika skor 0, tetapkan sebagai neutral
        polarity = 'neutral'

    return score, polarity

results = df['text_stopword'].apply(sentiment_analysis_lexicon_indonesia)
results = list(zip(*results))
df['polarity_score'] = results[0]
df['polarity'] = results[1]
print(df['polarity'].value_counts())

polarity
negative    33404
positive    30957
neutral      5828
Name: count, dtype: int64


In [13]:
vocab_size = 5000
embedding_dim = 100
max_length = 120
trunc_type = 'post'
padding_type = 'post'
oov_tok = "<OOV>"
training_portion = .8

In [14]:
# Pisahkan data menjadi fitur (tweet) dan label (sentimen)
X = df['text_akhir'].astype(str)
y = df['polarity']

In [15]:
# Bagi data latih dan uji (80:20)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [16]:
tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(X_train)

In [17]:
# Ubah teks menjadi sequences pada data training dan testing
X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)


In [18]:
# Lakukan padding pada sequences data training dan testing
X_train_pad = pad_sequences(X_train_seq, maxlen=max_length, padding=padding_type, truncating=trunc_type)
X_test_pad = pad_sequences(X_test_seq, maxlen=max_length, padding=padding_type, truncating=trunc_type)


In [19]:
# One-Hot Encoding untuk label
one_hot_encoder = OneHotEncoder(sparse_output=False)
y_train = one_hot_encoder.fit_transform(np.array(y_train).reshape(-1, 1))
y_test = one_hot_encoder.transform(np.array(y_test).reshape(-1, 1))


In [20]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    # tf.keras.layers.LSTM(64),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(128, return_sequences=True)), # Bidirectional LSTM
    # tf.keras.layers.Dropout(0.2), # Dropout for regularization
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)), # Another Bidirectional LSTM
    # tf.keras.layers.Dropout(0.2), # Dropout for regularization
    tf.keras.layers.Dense(64, activation='relu'), # Dense layer with ReLU
    tf.keras.layers.Dense(3, activation='softmax') # Output layer, sigmoid for binary classification
])

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [21]:
# Kompilasi model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [22]:
# Build model sebelum summary
model.build(input_shape=(None, max_length))
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding (Embedding)                │ (None, 120, 100)            │         500,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional (Bidirectional)        │ (None, 120, 256)            │         234,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional_1 (Bidirectional)      │ (None, 128)                 │         164,352 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 64)                  │           8,256 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 3)                   │             195 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 907,299 (3.46 MB)

 Trainable params: 907,299 (3.46 MB)

 Non-trainable params: 0 (0.00 B)

In [23]:
# Hitung bobot kelas berdasarkan distribusi data
class_weights = class_weight.compute_class_weight(
    class_weight='balanced',
    classes=np.unique(df['polarity']),  # Sesuaikan dengan label asli sebelum encoding
    y=df['polarity']
)

# Konversi ke dictionary agar bisa dipakai di model.fit()
class_weights_dict = dict(enumerate(class_weights))
print("Bobot kelas:", class_weights_dict)

Bobot kelas: {0: 0.7004051411008662, 1: 4.014470372912377, 2: 0.7557687545089425}


In [24]:
class EarlyStoppingCallback(tf.keras.callbacks.Callback):
        def on_epoch_end(self, epoch, logs=None):
            if logs.get('accuracy') > 0.92 and logs.get('val_accuracy') > 0.92:
                self.model.stop_training = True
                print("\nReached > 92% accuracy")

In [35]:
history = model.fit(
    X_train_pad,
    y_train,  # Use the original y_train that is already one-hot encoded
    epochs=20,
    batch_size=32,
    validation_data=(X_test_pad, y_test),
    class_weight=class_weights_dict,
    verbose=1,
    callbacks=[EarlyStoppingCallback()]
)

Epoch 1/20
1755/1755 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - accuracy: 0.9484 - loss: 0.1512
Reached > 92% accuracy
1755/1755 ━━━━━━━━━━━━━━━━━━━━ 40s 23ms/step - accuracy: 0.9484 - loss: 0.1512 - val_accuracy: 0.9264 - val_loss: 0.2191


In [37]:
# Contoh teks baru yang ingin diprediksi
teks_baru = "mudah digunakan"

# Praproses teks baru
teks_baru_clean = cleaningText(teks_baru)
teks_baru_casefolding = casefoldingText(teks_baru_clean)
teks_baru_slang = fix_slangwords(teks_baru_casefolding)
teks_baru_token = tokenizingText(teks_baru_slang)
teks_baru_stopword = filteringText(teks_baru_token)
teks_baru_akhir = toSentence(teks_baru_stopword)

# Ubah teks baru ke dalam sequence
teks_baru_seq = tokenizer.texts_to_sequences([teks_baru_akhir])

# Lakukan padding pada sequence
teks_baru_pad = pad_sequences(teks_baru_seq, maxlen=max_length, padding=padding_type, truncating=trunc_type)

# Lakukan prediksi
prediksi = model.predict(teks_baru_pad)

# Dapatkan kelas dengan probabilitas tertinggi
kelas_prediksi = np.argmax(prediksi)

# Ubah kelas prediksi ke label sentimen (positive, negative, atau neutral)
labels = ['negative', 'neutral', 'positive']  # Sesuaikan dengan urutan label pada one-hot encoding
sentimen_prediksi = labels[kelas_prediksi]

print(f"Teks: {teks_baru}")
print(f"Sentimen Prediksi: {sentimen_prediksi}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
Teks: mudah digunakan
Sentimen Prediksi: positive


### Mencoba algoritma lain

In [47]:
# Ekstraksi fitur dengan TF-IDF
tfidf = TfidfVectorizer(max_features=5000, min_df=5, max_df=0.8)
X_tfidf = tfidf.fit_transform(X)

In [48]:
# Konversi hasil ekstraksi fitur menjadi dataframe
features_df = pd.DataFrame(X_tfidf.toarray(), columns=tfidf.get_feature_names_out())

In [40]:
# Menampilkan hasil ekstraksi fitur
features_df

,admin,akun,alamat,alasan,aman,aneh,apk,aplikasi,aplikasinya,app,...,transaksi,trus,uang,udah,udh,ulasan,update,utk,voucher,yg
0,0.000000,0.000000,0.0,0.0,0.0000,0.0,0.0,0.000000,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.138589
1,0.000000,0.000000,0.0,0.0,0.0000,0.0,0.0,0.000000,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000
2,0.000000,0.147861,0.0,0.0,0.0000,0.0,0.0,0.098727,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.133209,0.0,0.0,0.189882
3,0.000000,0.000000,0.0,0.0,0.0000,0.0,0.0,0.000000,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000
4,0.572068,0.000000,0.0,0.0,0.0000,0.0,0.0,0.000000,0.0,0.0,...,0.202525,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70184,0.000000,0.000000,0.0,0.0,0.0000,0.0,0.0,0.000000,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000
70185,0.000000,0.000000,0.0,0.0,0.0000,0.0,0.0,0.000000,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.296197
70186,0.000000,0.000000,0.0,0.0,0.5861,0.0,0.0,0.000000,0.0,0.0,...,0.418901,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.303341
70187,0.000000,0.000000,0.0,0.0,0.0000,0.0,0.0,0.000000,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000


In [49]:
from sklearn.linear_model import LogisticRegression

# Bagi data menjadi data latih dan data uji (80:20)
X_train, X_test, y_train, y_test = train_test_split(X_tfidf, y, test_size=0.2, random_state=42)

# Inisialisasi model Logistic Regression
logistic_regression = LogisticRegression()

# Latih model dengan data latih
logistic_regression.fit(X_train, y_train)

# Prediksi pada data latih dan data uji
y_pred_train_lr = logistic_regression.predict(X_train)
y_pred_test_lr = logistic_regression.predict(X_test)

# Evaluasi akurasi
accuracy_train_lr = accuracy_score(y_train, y_pred_train_lr)
accuracy_test_lr = accuracy_score(y_test, y_pred_test_lr)

# Print hasil akurasi
print('Logistic Regression with TF-IDF - accuracy_train:', accuracy_train_lr)
print('Logistic Regression with TF-IDF - accuracy_test:', accuracy_test_lr)

Logistic Regression with TF-IDF - accuracy_train: 0.9226193656390803
Logistic Regression with TF-IDF - accuracy_test: 0.9048297478273258


In [50]:
# Bagi data menjadi data latih dan data uji (80:20)
X_train, X_test, y_train, y_test = train_test_split(X_tfidf, y, test_size=0.2, random_state=42)

# Inisialisasi model Naive Bayes
naive_bayes = MultinomialNB()

# Latih model dengan data latih
naive_bayes.fit(X_train, y_train)

# Prediksi pada data latih dan data uji
y_pred_train_nb = naive_bayes.predict(X_train)
y_pred_test_nb = naive_bayes.predict(X_test)

# Evaluasi akurasi
accuracy_train_nb = accuracy_score(y_train, y_pred_train_nb)
accuracy_test_nb = accuracy_score(y_test, y_pred_test_nb)

# Print hasil akurasi
print('Naive Bayes with TF-IDF - accuracy_train:', accuracy_train_nb)
print('Naive Bayes with TF-IDF - accuracy_test:', accuracy_test_nb)

Naive Bayes with TF-IDF - accuracy_train: 0.7627468789513989
Naive Bayes with TF-IDF - accuracy_test: 0.7426271548653655


In [44]:
# Tokenize the text
tokenized_reviews = df['text_akhir'].apply(lambda x: x.split())

# Train Word2Vec model
model_w2v = Word2Vec(sentences=tokenized_reviews, vector_size=100, window=5, min_count=1, workers=4)

# Function to create vector representation of a review
def vectorize_review(review):
    vectors = []
    for word in review:
        if word in model_w2v.wv:
            vectors.append(model_w2v.wv[word])
    if vectors:
        return np.mean(vectors, axis=0)
    else:
        return np.zeros(100)

# Create feature vectors
X_w2v = np.array([vectorize_review(review) for review in tokenized_reviews])

# Split data
X_train_w2v, X_test_w2v, y_train_w2v, y_test_w2v = train_test_split(X_w2v, y, test_size=0.2, random_state=42)


In [45]:
# SVM + Word2Vec 80:20
# Bagi data menjadi data latih dan data uji

# Membuat objek model SVM
svm_model = SVC()

# Melatih model SVM pada data pelatihan
svm_model.fit(X_train_w2v, y_train_w2v)

# Prediksi sentimen pada data uji
y_pred_svm_train_w2v = svm_model.predict(X_train_w2v)
y_pred_svm_test_w2v = svm_model.predict(X_test_w2v)

# Evaluasi akurasi model SVM
accuracy_train_svm_w2v = accuracy_score(y_train_w2v, y_pred_svm_train_w2v)
accuracy_test_svm_w2v = accuracy_score(y_test_w2v, y_pred_svm_test_w2v)

print('SVM with Word2Vec - accuracy_train:', accuracy_train_svm_w2v)
print('SVM with Word2Vec - accuracy_test:', accuracy_test_svm_w2v)


SVM with Word2Vec - accuracy_train: 0.7498530747448844
SVM with Word2Vec - accuracy_test: 0.7389941587120672


In [54]:
# Membuat objek model Random Forest
random_forest = RandomForestClassifier(
    n_estimators=100,  # Jumlah pohon dalam ensemble
    max_depth=None,  # Kedalaman maksimum pohon
    min_samples_split=2,  # Jumlah sampel minimum untuk membagi node
    min_samples_leaf=1
)

# Melatih model Random Forest pada data pelatihan
random_forest.fit(X_train, y_train)

# Prediksi sentimen pada data pelatihan dan data uji
y_pred_train_rf = random_forest.predict(X_train)
y_pred_test_rf = random_forest.predict(X_test)

# Evaluasi akurasi model Random Forest
accuracy_train_rf = accuracy_score(y_pred_train_rf, y_train)
accuracy_test_rf = accuracy_score(y_pred_test_rf, y_test)

# Menampilkan akurasi
print('Random Forest with TF-IDF - accuracy_train:', accuracy_train_rf)
print('Random Forest with TF-IDF - accuracy_test:', accuracy_test_rf)

Random Forest with TF-IDF - accuracy_train: 0.9998753361471746
Random Forest with TF-IDF - accuracy_test: 0.8170679583986323
